# Gaussian Process Mean-variance trading with signatures


In [1]:
from functools import partial, reduce
from collections import OrderedDict

import scipy 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import mogptk  # pytorch >= 11.0 is required
import torch # pytorch 11.0 is used
import signatory # if cannot find a version compatible with pytorch, 
                 # install locally follows insturcction on https://github.com/patrick-kidger/signatory

from src.signature_trading import SignatureTrading,trading_strategies,GaussianProcessExpectedSiganture,_transformation
from src.free_lie_algebra import *
from src.transformations import LeadLag, HoffLeadLag, AddTime, ScalePaths, TranslatePaths
from src.optimisation.signature import *
from src.helper_functions.signature_helper_functions import get_signature_weights, \
    get_signature_values

ModuleNotFoundError: No module named 'yfinance'

In [ ]:
# Get data
_N = 4

df      = pd.read_csv('stocks.csv', index_col=0)
names   = df.columns[:_N].to_list()
df.index = pd.to_datetime(df.index)

df = df.dropna()

names            = df.columns[:_N].to_list()
indexes          = df.index.to_numpy()

## Basic setup for signature trading:

In [ ]:
mv_result_df =pd.DataFrame(columns= ['Capital','PnL','weight GE','weight IBM','weight JPM','weight KO'])

In [ ]:
df2 = df[names].loc[(df.index > pd.Timestamp('2017-01-01')) & (df.index < pd.Timestamp('2022-01-01'))]

We may first look at the traditional Markowitz mean-variance optimisation.

In [ ]:
for i in range(len(df2)-28+1):
    if len(df2)-5*i <28:
        break
    df3 = df2[5*i:28+5*i]
    prices = df3.to_numpy()
    df3['time'] = np.linspace(0,1,len(df3))
    df_train = df3[:23]
    r = (prices[:-1]-prices[1:])/prices[1:]
    train_r = r[:22]
    test_r = r[22:]
    df_test = df3[names+['time']][22:]
    if i==0:
        start_date = df_test.index[0]
    start_capital = capital_mv[-1]
    expt_mean,expt_cov = np.mean(train_r,axis=0),np.cov(train_r.T)
    mv_weights = np.array([
            mean_variance_optim(expt_mean,expt_cov, pnl) for pnl in np.linspace(0,0.1,50)
         ])
    risks = [np.matmul(w.T, np.matmul(expt_cov, w)) for w in mv_weights]
    pnls = [np.matmul(w.T, expt_mean) for w in mv_weights]
    weight_mv = mv_weights[np.argmin(risks)]
    for j in range(len(test_r)):
        mv_result = []
        practical_pnl = np.matmul(weight_mv.T, test_r[j])

        capital_mv.append(capital_mv[-1]*(1+practical_pnl))
        mv_result.append(capital_mv[-1])
        mv_result.append(practical_pnl)
        mv_result += list(weight_mv)
        mv_result_df.loc[df_test.index[j+1]] = mv_result

## Backtest:

We backtest the performance with stock 'GE','IBM','JPM' and 'KO' from 2017-01-01 to 2022-01-01. We also compute weights of each asset of Markowitz mean-variance portofolio with empirical mean and variance of assets and mean variance estimated from the fitted Gaussain Process model.The PnL and value of portofolios are recoreded as well as benchmarks.

In [ ]:
capital_mv = [100000]

for i in range(len(df2)-28+1):
    if len(df2)-5*i <28:
        break
    df3 = df2[5*i:28+5*i]
    prices = df3.to_numpy()
    df3['time'] = np.linspace(0,1,len(df3))
    df_train = df3[:23]
    r = (prices[:-1]-prices[1:])/prices[1:]
    train_r = r[:22]
    test_r = r[22:]
    df_test = df3[names+['time']][22:]
    if i==0:
        start_date = df_test.index[0]
    start_capital = capital_mv[-1]
    expt_mean,expt_cov = np.mean(train_r,axis=0),np.cov(train_r.T)
    mv_weights = np.array([
            mean_variance_optim(expt_mean,expt_cov, pnl) for pnl in np.linspace(0,0.1,50)
         ])
    risks = [np.matmul(w.T, np.matmul(expt_cov, w)) for w in mv_weights]
    pnls = [np.matmul(w.T, expt_mean) for w in mv_weights]
    weight_mv = mv_weights[np.argmin(risks)]
    for j in range(len(test_r)):
        mv_result = []
        practical_pnl = np.matmul(weight_mv.T, test_r[j])

        capital_mv.append(capital_mv[-1]*(1+practical_pnl))
        mv_result.append(capital_mv[-1])
        mv_result.append(practical_pnl)
        mv_result += list(weight_mv)
        mv_result_df.loc[df_test.index[j+1]] = mv_result

In [ ]:
def training_lmc(data,Q=3,init_method = 'BNSE',method = 'Adam',lr = 0.01,iters = 500,training_size=23,training_in_gpu=0):
        """
        Return a trained lmc model for given data
        
        Arugments:
        ---------------
        backtest=False: bool
            mode that is backtest friendly
        training_size: float/int
            set the size of training set
        For the other arguments, we refer [1] for a detailed explanation

        Reference:
        --------------
        [1]  T. de Wolff, A. Cuevas, and F. Tobar. MOGPTK: The Multi-Output Gaussian Process Toolkit. Neurocomputing, 2020
        """
        # Default training size is 23 if not specified
        if training_in_gpu!=None and torch.cuda.is_available():
            mogptk.use_gpu(training_in_gpu)
        elif not torch.cuda.is_available():
            print("CUDA is not available.")
        else:
            mogptk.use_cpu()
        lmc_dataset = mogptk.LoadDataFrame(data, x_col='time', y_col=names)
        for channel in lmc_dataset:
            channel.transform(mogptk.TransformNormalize())
            channel.transform(mogptk.TransformDetrend())
        for name in names:
            # remove the data after `training_size` as test data 
            lmc_dataset[name].remove_range(data.time[training_size],None)

        lmc = mogptk.SM_LMC(lmc_dataset, Q=Q,inference=mogptk.Exact())
        lmc.init_parameters(init_method)
        lmc.train(method=method, lr=lr, iters=iters, verbose=False, error='MSE')
        return lmc

#dt = timedelta_to_fraction_of_year(np.diff(indexes).mean())
# set the transofrmation
transformations = _transformation(_N,dt=None,_transformations=OrderedDict(
                {"AddTime": False, "TranslatePaths": False, "ScalePaths": False, "LeadLag": False, "HoffLeadLag":True}))

In [ ]:
sig_result_df = pd.DataFrame(columns= ['Capital','PnL','weight GE','weight IBM','weight JPM','weight KO'])
gmv_result_df = pd.DataFrame(columns= ['Capital','PnL','weight GE','weight IBM','weight JPM','weight KO'])
initial_capital = 100000
capital_sig = [initial_capital]
capital_gmv = [initial_capital]
for i in range(len(df2)-28+1):
    if len(df2)-5*i <28:
        break
    df3 = df2[5*i:28+5*i]
    df_train = df3[:23]
    
    gpm = training_lmc(df3)
    ges = GaussianProcessExpectedSiganture(gpm)
    ges._get_paths()
    sig_trading = SignatureTrading(df3,ges,level=2)
    sig_trading._get_funcs(transformation=transformations)
    test_time = df_test.time.to_numpy()

    price = df_test.iloc[:,:_N].to_numpy()

    for j in range(len(test_time)-1):
        result = []
        gmv_result = []

        es_weights = sig_trading.get_weights(price[j])
        weight = es_weights/np.sum(es_weights)

        shares = capital_sig[-1]*weight/price[j]
                
        capital_sig.append(np.sum(shares * price[j+1]))

        result.append(capital_sig[-1])
        result.append((capital_sig[-1]-capital_sig[-2])/capital_sig[-2])
        result += list(weight)
        
        expt_mean,expt_cov = gpm.gpr.predict(torch.tensor([[i,test_time[j]] for i in range(4)]),full=True)
        gmv_weights = np.array([
            mean_variance_optim(expt_mean,expt_cov, pnl) for pnl in np.linspace(0,0.1,50)
         ])

        pnls = [np.matmul(w.T, expt_mean) for w in gmv_weights]

        risks = [np.matmul(w.T, np.matmul(expt_cov, w)) for w in gmv_weights]
        weight_gmv = gmv_weights[np.argmin(risks)]
        shares_gmv = capital_gmv[-1]*weight_gmv/price[j]
        capital_gmv.append(np.sum(shares_gmv * price[j+1]))
        gmv_result.append(capital_gmv[-1])
        gmv_result.append((capital_gmv[-1]-capital_gmv[-2])/capital_gmv[-2])
        gmv_result += list(weight_gmv)

        sig_result_df.loc[df_test.index[j+1]] = result
        gmv_result_df.loc[df_test.index[j+1]] = gmv_result

In [ ]:
mv_result_df.to_csv("mv_result.csv")
sig_result_df.to_csv("sig_result.csv")
gmv_result_df.to_csv("gmv_result.csv")

Here is the daily PnL of signature trading. We may see a big drop at the beginning of 2021. 

In [ ]:
plt.title("Daily PnL with signature trading strategy")
plt.xlabel("Date")
plt.ylabel("Daily PnL")
plt.plot(gmv_result_df.PnL,label='gaussian mean-variance')
plt.plot(mv_result_df.PnL,label='mean-variance')
plt.plot(sig_result_df.PnL,label='signature trading')
plt.legend()

Let's have a look at the drop and see what happens,

In [ ]:
df2 = df1[names].loc[(df1.index > pd.Timestamp('2020-02-08')) & (df1.index < pd.Timestamp('2020-03-20'))]

In [ ]:
df3 = df2[0:28]
df_train = df3[:23]
df_test = df3[names][22:]

ges = GaussianProcessExpectedSiganture(training_lmc(df3))
ges._get_paths()
sig_trading = SignatureTrading(df3,ges,level=2)
sig_trading._get_coeffs(transformation=transformations)
expected_signatures_ = np.array([get_signature_values(sig_trading.normed_price[np.newaxis, :i, :], 2) \
                                 for i in range(22,len(df3)-1)])

We may see that the logarithm of the absolute value of the 'optimal' variance of the portofolio is negative with a large magnitude. It is clear that the optimization algorithm fails on the time period. 

In [ ]:
_,sig_var = sig_trading._get_coeffs()

In [ ]:
plt.title("log(-var) of the optimal variance trained in the period 2020-02-10 to 2020-03-19")
plt.plot(np.log(-sig_var))
plt.ylabel("log(-var)")

All of the four assets are predicted to show a decreasing trend. This 'wrong' way correlation and an inaccurate approximation of signatures cause this failure of convergence of the optimization algorithm.

In [ ]:
sig_trading.es.lmc.plot_prediction()

In [ ]:
df2 = df[names].loc[(df.index > pd.Timestamp('2021-11-18')) & (df.index <= pd.Timestamp('2021-12-30'))]

Now we apply the ReLU function on the weights and then normalized them to get a new set of weights that are between 0 and 1. Below is the plot of how this would affect the strategy, and we can see that extreme values are obviously reduced.

In [ ]:
noshorting_sig_result_df = pd.DataFrame(columns= ['Capital','PnL','weight GE','weight IBM','weight JPM','weight KO'])
capital_nssig = [100000]
for i in range(len(df2)-28+1):
    if len(df2)-5*i <28:
        break
    df3 = df2[5*i:28+5*i]

    df3['time'] = np.linspace(0,1,len(df3))
    df_train = df3[:23]
    df_test = df3[names+['time']][22:]
    price = df_test.iloc[:,:_N].to_numpy()
    for j in range(len(df_test)-1):
        result = []
        if df_test.index[j] in sig_result_df.index:
            es_weights = sig_result_df.loc[df_test.index[j],\
                                        ['weight GE','weight IBM','weight JPM','weight KO']].to_numpy()
            pos_weight = es_weights*np.array(es_weights>0,dtype=np.float64)
            weight = pos_weight/np.sum(pos_weight)

            shares = capital_nssig[-1]*weight/price[j]

            capital_nssig.append(np.sum(shares * price[j+1]))

            result.append(capital_nssig[-1])
            result.append((capital_nssig[-1]-capital_nssig[-2])/capital_nssig[-2])

            result += list(weight)
            noshorting_sig_result_df.loc[df_test.index[j+1]] = result

In [ ]:
plt.title("Daily PnL with signature trading strategy vs no shorting")
#plt.plot(result_df[result_df8.PnL>-0.2].PnL,label='signature trading')
plt.plot(sig_result_df.PnL,label='signature trading')
plt.plot(noshorting_sig_result_df.PnL,label='siganture trading without shorting',alpha=0.5)
plt.legend()
plt.xlabel("Date")
plt.ylabel("Daily PnL")

Finally, we compare the signature trading strategy without shorting to (Gaussain) Markowitz mean-variance portofolio, and we can see that the PnL indeed becomes more stable.

In [ ]:
plt.plot(mv_result_df.PnL,label='usual markowitz')
plt.plot(gmv_result_df.PnL,label='gaussian process markowitz')
#plt.plot(result_df3[result_df3.PnL>-0.2].PnL)
plt.plot(noshorting_sig_result_df.PnL,label='signature')
plt.title("Daily PnL with different trading strategies")
plt.xlabel("Date")
plt.ylabel("Daily PnL")
plt.legend()

In [ ]:
noshorting_sig_result_df.to_csv("no shorting.csv")

Below is the plot of change of portofolio values. 

In [ ]:
plt.title("Dality Portfolio values for three different strategies")
plt.plot(noshorting_sig_result_df.Capital,label='signature trading without shorting')
plt.plot(sig_result_df.Capital,label='signature trading')
plt.plot(gmv_result_df.Capital,label='gaussian process markowitz')
plt.plot(mv_result_df.Capital,label='usual markowitz')
plt.xlabel("Date")
plt.ylabel("Capital")
plt.legend()